# OpenFDA Manufacturer - Sunshine Act Manufacturer Matching
## Authors: 
    1. Lam Ho
    2. Jonah Breslow
    3. Jeffrey Kagan
## Purpose:
The purpose of this notebook is to match the manufacturer entities from the OpenFDA data to the manufacturer entities in the Sunshine Act data. For this procedure, we utilized the Dedupe.io python implementation.

### Importing modules

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""
This code demonstrates how to use RecordLink with two comma separated
values (CSV) files. We have listings of products from two different
online stores. The task is to link products between the datasets.

The output will be a CSV with our linkded results.

"""
import os
import csv
import re
import logging
import optparse
import pandas as pd
import numpy as np
import pickle

import dedupe
from unidecode import unidecode

### Importing Data

In [2]:
sunshine_manuf = pd.read_csv('Data/Outputs_Cleanup/Sunshine/pharmCo_info.csv')
df = sunshine_manuf[['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name']]
df.columns = ['manufacturer_name']
df.to_csv('Data/Outputs_Cleanup/Sunshine/pharmCo_manufacturer_dedupe_input.csv', index=False)
#df.head()

In [3]:
# openfda_manuf = pd.read_csv('Data/Outputs_Cleanup/FDA/openfda_manufacturer_deduplicated_single_manuf.csv')#df = openfda_manuf[['manufacturer_name']]
openfda_manuf = pickle.load(open("Data/Outputs_Cleanup/FDA/openfda_manufacturer_deduplicated_single_manuf.p", "rb" )).reset_index()
df = openfda_manuf[['manufacturer_name']]
#df.columns = ['manufacturer_name']
df.to_csv('Data/Outputs_Cleanup/FDA/openfda_manufacturer_dedupe_input.csv', index=False)

### Running the Dedupe procedure

In [4]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """

    column = unidecode(column)
    column = re.sub('\n', ' ', column)
    column = re.sub('-', '', column)
    column = re.sub('/', ' ', column)
    column = re.sub("'", '', column)
    column = re.sub(",", '', column)
    column = re.sub(":", ' ', column)
    column = re.sub('  +', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column:
        column = None
    return column


def readData(filename):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID.
    """

    data_d = {}

    with open(filename) as f:
        reader = csv.DictReader(f)
        for i, row in enumerate(reader):
            clean_row = dict([(k, preProcess(v)) for (k, v) in row.items()])
            data_d[filename + str(i)] = dict(clean_row)

    return data_d

## Parameters

In [5]:
retrain = False

In [6]:
# Parameters
retrain = False


In [7]:
 # ## Setup
output_file = 'Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_output.csv'
settings_file = 'Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_learned_settings'
training_file = 'Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_training.json'

if retrain == True:
    try:
        os.remove(settings_file)
    except:
        print('Your settings file appears to not have existed.')

left_file = 'Data/Outputs_Cleanup/Sunshine/pharmCo_manufacturer_dedupe_input.csv'
right_file = 'Data/Outputs_Cleanup/FDA/openfda_manufacturer_dedupe_input.csv'

print('importing data ...')
data_1 = readData(left_file)
data_2 = readData(right_file)

def descriptions():
    for dataset in (data_1, data_2):
        for record in dataset.values():
            yield record['description']

# ## Training

if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as sf:
        linker = dedupe.StaticRecordLink(sf)

else:
    # Define the fields the linker will pay attention to
    #
    # Notice how we are telling the linker to use a custom field comparator
    # for the 'price' field.
    fields = [
        {'field': 'manufacturer_name', 'type': 'String'},
        #{'field': 'title', 'type': 'Text', 'corpus': descriptions()},
        #{'field': 'description', 'type': 'Text',
        # 'has missing': True, 'corpus': descriptions()},
        #{'field': 'price', 'type': 'Price', 'has missing': True}
    ]

    # Create a new linker object and pass our data model to it.
    linker = dedupe.RecordLink(fields)

    # If we have training data saved from a previous run of linker,
    # look for it an load it in.
    # __Note:__ if you want to train from scratch, delete the training_file
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file) as tf:
            linker.prepare_training(data_1,
                                    data_2,
                                    training_file=tf,
                                    sample_size=15000)
    else:
        linker.prepare_training(data_1, data_2, sample_size=15000)

    # ## Active learning
    # Dedupe will find the next pair of records
    # it is least certain about and ask you to label them as matches
    # or not.
    # use 'y', 'n' and 'u' keys to flag duplicates
    # press 'f' when you are finished
    print('starting active labeling...')

    dedupe.console_label(linker)

    linker.train()

    # When finished, save our training away to disk
    with open(training_file, 'w') as tf:
        linker.write_training(tf)

    # Save our weights and predicates to disk.  If the settings file
    # exists, we will skip all the training and learning next time we run
    # this file.
    with open(settings_file, 'wb') as sf:
        linker.write_settings(sf)

# ## Blocking

# ## Clustering

# Find the threshold that will maximize a weighted average of our
# precision and recall.  When we set the recall weight to 2, we are
# saying we care twice as much about recall as we do precision.
#
# If we had more data, we would not pass in all the blocked data into
# this function but a representative sample.

print('clustering...')
linked_records = linker.join(data_1, data_2, 0.0)

print('# duplicate sets', len(linked_records))
# ## Writing Results

# Write our original data back out to a CSV with a new column called
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership = {}
for cluster_id, (cluster, score) in enumerate(linked_records):
    for record_id in cluster:
        cluster_membership[record_id] = {'Cluster ID': cluster_id,
                                         'Link Score': score}

with open(output_file, 'w') as f:

    header_unwritten = True

    for fileno, filename in enumerate((left_file, right_file)):
        with open(filename) as f_input:
            reader = csv.DictReader(f_input)

            if header_unwritten:

                fieldnames = (['Cluster ID', 'Link Score', 'source file'] +
                              reader.fieldnames)

                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()

                header_unwritten = False

            for row_id, row in enumerate(reader):

                record_id = filename + str(row_id)
                cluster_details = cluster_membership.get(record_id, {})
                row['source file'] = fileno
                row.update(cluster_details)

                writer.writerow(row)

importing data ...
reading from Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_learned_settings


INFO:dedupe.api:Predicate set:


INFO:dedupe.api:(SimplePredicate: (sameSevenCharStartPredicate, manufacturer_name), TfidfNGramSearchPredicate: (0.2, manufacturer_name))


INFO:dedupe.api:(SimplePredicate: (sameFiveCharStartPredicate, manufacturer_name), SimplePredicate: (commonTwoTokens, manufacturer_name))


clustering...


...

INFO:dedupe.canopy_index:Removing stop word ed


INFO:dedupe.canopy_index:Removing stop word ro


INFO:dedupe.canopy_index:Removing stop word  d


INFO:dedupe.canopy_index:Removing stop word at


# duplicate sets 385


### Post-Processing the Manufacturer Matches

In [8]:
# Reading in Dedupe output
df = pd.read_csv('Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_output.csv')
# Splitting out by input file
df_fda = df[df['source file'] == 1]
df_sunshine = df[df['source file'] == 0]

# Loading the keys that point to the source data
df_fda = df_fda.merge(openfda_manuf[['manufacturer_name', 'fda_manuf_id']], how='left', on='manufacturer_name')
df_sunshine = df_sunshine.merge(sunshine_manuf,'left',left_on='manufacturer_name',right_on='Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name')
df_sunshine.drop(columns = ['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'],inplace=True)


manuf_merge = pd.concat([df_fda, df_sunshine], axis=0).sort_values('Cluster ID')
manuf_merge['fda_manuf_id']  = manuf_merge['fda_manuf_id'].fillna('[]')
manuf_merge['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID'] = manuf_merge['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID'].fillna('[]')
# manuf_merge

### Aggregating Matches

In [9]:
# splitting out the matches and the unmatches
manuf_merge_matched= manuf_merge[~manuf_merge['Cluster ID'].isna()]
manuf_merge_unmatched= manuf_merge[manuf_merge['Cluster ID'].isna()]

# Aggregating all the matches into a single record (list)
manuf_merge_matched = manuf_merge_matched.groupby('Cluster ID').agg(list)

### Choosing Manufacturer Name

In [10]:
# UDF that picks the non empty list within a list
def pick_list(x):
    my_pick = None
    for each in x:
        if each == '[]':
            continue
        else:
            my_pick = each
    return my_pick

manuf_merge_matched['fda_manuf_id'] = manuf_merge_matched['fda_manuf_id'].apply(lambda x: pick_list(x))
manuf_merge_matched['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID'] = manuf_merge_matched['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID'].apply(lambda x: pick_list(x))

In [11]:
# UDF that picks the longest name
def pick_longest(x):
    longest = None
    longest_len = 0
    for idx, each in enumerate(x):
        if idx == 0:
            longest = each
            longest_len = len(each)
            continue
        if len(each) > longest_len:
            longest = each
            longest_len = len(each)  
    return longest

manuf_merge_matched['manufacturer_name'] = manuf_merge_matched['manufacturer_name'].apply(lambda x: pick_longest(x))
manuf_merge_matched = manuf_merge_matched[['manufacturer_name', 'fda_manuf_id', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID']]
manuf_merge = pd.concat([manuf_merge_matched,manuf_merge_unmatched])
manuf_merge = manuf_merge[['manufacturer_name', 'fda_manuf_id', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID']]

### Creating `manuf_id`

In [12]:
# # Creating a matched ID
manuf_merge.reset_index(inplace=True)
manuf_merge.drop(columns = ['index'],inplace=True)
manuf_merge.reset_index(inplace=True)
manuf_merge.rename(columns = {'index':'manuf_id'},inplace=True)

In [13]:
manuf_merge['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID'] =(
    np.where(manuf_merge['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID']=='[]',
             np.NaN,
             manuf_merge['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID'] 
            )
)

manuf_merge['fda_manuf_id'] =(
    np.where(manuf_merge['fda_manuf_id']=='[]',
             np.NaN,
             manuf_merge['fda_manuf_id'] 
            )
)

### Exporting Data

In [14]:
pickle.dump(manuf_merge, open( "Data/Outputs_Cleanup/Manufacturer_entity_matching/Manufacturer_Matching_lookup.p", "wb" ) )